In [315]:
from function import *
import numpy as np
import networkx as nx
import time
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import matplotlib.pyplot as plt

#### LOAD THE DATA

In [329]:
# Using Random Data
# N = 100000                # Number of data
# d = 2                     # Number of features
# precision = 2             # Control precision bit of the output
# dim = d * (2 * precision) # Size of the QUBO matrix
# degree = 2                # Degree of the polynomial regression

# # Generate Random Data
# X = np.random.rand(N, d)
# Y = np.random.rand(N)

# Using Dataset
start_time = time.time()

dataset_path = 'dataset/processed dataset/dataset_9.csv'
dataset_name = dataset_path.split('/')[-1]
df = pd.read_csv(dataset_path)

X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values
d = X.shape[1]
precision = 3
dim = d * (2 * precision)
degree = 2

# Train-Test data splitting
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# If you want to perform polynomial regression
X_train, d, dim = polynomialForm(X_train, d, dim, precision, degree)
X_test, _, _ = polynomialForm(X_test, d, dim, precision, degree) 

end_time = time.time()
preproccesing1_time = end_time - start_time

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(17010, 3403)
(4253, 3403)
(17010,)
(4253,)


#### COMPUTING SOME IMPORTANT VARIABLES
Initiating variables that required to creating QUBO Matrix

In [317]:
start_time = time.time()

XtX = np.matmul(X_train.T, X_train) # Covariance matrix
XtY = np.matmul(X_train.T, Y_train) # Relation between Features and Y

end_time = time.time()
preproccesing2_time = end_time - start_time

print(XtX)
print(XtY)

[[1.63942400e+06 1.45892852e+05 9.34460009e+05 ... 1.78961093e+03
  2.67098206e+03 7.00606816e+04]
 [1.45892852e+05 2.37670451e+04 8.15951070e+04 ... 2.46795282e+02
  3.87996651e+02 7.50883044e+03]
 [9.34460009e+05 8.15951070e+04 5.50590568e+05 ... 7.90300625e+02
  1.25555663e+03 3.74588157e+04]
 ...
 [1.78961093e+03 2.46795282e+02 7.90300625e+02 ... 2.07130116e+02
  1.92215979e+02 2.17818361e+02]
 [2.67098206e+03 3.87996651e+02 1.25555663e+03 ... 1.92215979e+02
  2.17818361e+02 3.95133883e+02]
 [7.00606816e+04 7.50883044e+03 3.74588157e+04 ... 2.17818361e+02
  3.95133883e+02 1.44702855e+04]]
[1.50747078e+05 1.65533701e+04 7.93626588e+04 2.82824928e+04
 7.22203982e+03 7.37908317e+03 5.86640456e+04 3.21235197e+03
 8.50275171e+03 3.56749667e+03 1.09164125e+03 1.07054090e+03
 6.49774833e+03 4.36106786e+04 1.38249925e+04 3.19281642e+03
 3.32988431e+03 3.01356774e+04 7.62919241e+03 2.72228997e+03
 2.74329435e+03 1.26721058e+04 3.03471822e+03 4.01140783e+02
 2.88260205e+03 3.45890179e+03 2.8

In [318]:
total_preproccesing_time = preproccesing1_time + preproccesing2_time

#### FIXSTAR AMPLIFY SIMULATED ANNEALER

In [319]:
start_time = time.time()
model = generateQuboMatrix_fixstar(XtX, XtY, dim, precision, d)
end_time = time.time()
fixstar_preproccesing_time = end_time - start_time + total_preproccesing_time

start_time = time.time()
sampleset_fixstar = sampling_fixstar(model)
end_time = time.time()
fixstar_execution_time = end_time - start_time

start_time = time.time()
distributions_fixstar, weights_fixstar = solve_fixstar(sampleset_fixstar, dim, precision, d)
end_time = time.time()
fixstar_postproccesing_time = end_time - start_time

In [320]:
Y_train_pred_fixstar = np.dot(X_train, weights_fixstar)
Y_test_pred_fixstar = np.dot(X_test, weights_fixstar)

# Evaluate on training data
train_mse_fixstar = mean_squared_error(Y_train, Y_train_pred_fixstar)
train_mae_fixstar = mean_absolute_error(Y_train, Y_train_pred_fixstar)
train_r2_fixstar = r2_score(Y_train, Y_train_pred_fixstar) 

# Evaluate on test data
test_mse_fixstar = mean_squared_error(Y_test, Y_test_pred_fixstar)
test_mae_fixstar = mean_absolute_error(Y_test, Y_test_pred_fixstar)
test_r2_fixstar = r2_score(Y_test, Y_test_pred_fixstar)

# Print the metrics
print("Fixstar Training Evaluation:")
print(f"MSE: {train_mse_fixstar:.5f}, MAE: {train_mae_fixstar:.5f}, R²: {train_r2_fixstar:.5f}")    

print("\nFixstar Testing Evaluation:")
print(f"MSE: {test_mse_fixstar:.5f}, MAE: {test_mae_fixstar:.5f}, R²: {test_r2_fixstar:.5f}")  

Fixstar Training Evaluation:
MSE: 0.00011, MAE: 0.00701, R²: 0.98852

Fixstar Testing Evaluation:
MSE: 0.00011, MAE: 0.00701, R²: 0.98859


#### NEAL SIMULATED ANNEALER

In [321]:
start_time = time.time()
Q = generateQuboMatrix_neal(XtX, XtY, precision, d)
end_time = time.time()
neal_preproccesing_time = end_time - start_time + total_preproccesing_time

start_time = time.time()
sampleset_neal = sampling_neal(Q)
end_time = time.time()
neal_execution_time = end_time - start_time 

start_time = time.time()
distributions_neal, weights_neal = solve_neal(sampleset_neal, dim, precision, d)
end_time = time.time()
neal_postproccesing_time = end_time - start_time 

In [322]:
Y_train_pred_neal = np.dot(X_train, weights_neal)
Y_test_pred_neal = np.dot(X_test, weights_neal)

# Evaluate on training data
train_mse_neal = mean_squared_error(Y_train, Y_train_pred_neal)
train_mae_neal = mean_absolute_error(Y_train, Y_train_pred_neal)
train_r2_neal = r2_score(Y_train, Y_train_pred_neal)

# Evaluate on test data
test_mse_neal = mean_squared_error(Y_test, Y_test_pred_neal)
test_mae_neal = mean_absolute_error(Y_test, Y_test_pred_neal)
test_r2_neal = r2_score(Y_test, Y_test_pred_neal)

# Print the metrics
print("Neal Training Evaluation:")
print(f"MSE: {train_mse_neal:.5f}, MAE: {train_mae_neal:.5f}, R²: {train_r2_neal:.5f}")

print("\nNeal Testing Evaluation:")
print(f"MSE: {test_mse_neal:.5f}, MAE: {test_mae_neal:.5f}, R²: {test_r2_neal:.5f}")

Neal Training Evaluation:
MSE: 0.00732, MAE: 0.05258, R²: 0.19963

Neal Testing Evaluation:
MSE: 0.00735, MAE: 0.05266, R²: 0.20179


#### SCIKIT LEARN (TRADISIONAL WAY)

In [323]:
start_time = time.time() 

clf = LinearRegression()
clf.fit(X_train, Y_train)
print("Coefficients:", clf.coef_)

Y_train_pred_sklearn = clf.predict(X_train)
Y_test_pred_sklearn = clf.predict(X_test)

# Evaluate on training data
train_mse_sklearn = mean_squared_error(Y_train, Y_train_pred_sklearn)
train_mae_sklearn = mean_absolute_error(Y_train, Y_train_pred_sklearn)
train_r2_sklearn = r2_score(Y_train, Y_train_pred_sklearn) 

# Evaluate on test data
test_mse_sklearn = mean_squared_error(Y_test, Y_test_pred_sklearn)
test_mae_sklearn = mean_absolute_error(Y_test, Y_test_pred_sklearn)
test_r2_sklearn = r2_score(Y_test, Y_test_pred_sklearn)

end_time = time.time()  
sklearn_execution_time = end_time - start_time

# Print the metrics
print("Fixstar Training Evaluation:")
print(f"MSE: {train_mse_sklearn:.5f}, MAE: {train_mae_sklearn:.5f}, R²: {train_r2_sklearn:.5f}")    

print("\nFixstar Testing Evaluation:")
print(f"MSE: {test_mse_sklearn:.5f}, MAE: {test_mae_sklearn:.5f}, R²: {test_r2_sklearn:.5f}")  

Coefficients: [ 8.63568656e-15  1.23565849e-02  7.24960084e-03  8.72281468e-01
 -1.04707450e-01  9.52238771e-03  1.84619254e-03 -1.94394123e-01
 -9.38977016e-02  2.63839933e-01 -2.15598254e-01  1.33250116e-02
  1.36704060e-02 -1.48587021e-02  3.89937625e-01 -2.03204012e-01
 -4.42126046e-02 -7.44789726e-02  6.40211341e-01 -5.30485700e-01
 -6.82057164e-01 -2.34872587e-01  6.95852977e-01  5.86202403e-01
  2.06673341e-01  2.01083361e-01  1.02795095e-01  1.01281361e-01
  1.70504618e-01 -5.61374430e-02  3.84111122e-01  1.50218262e-02
  1.08219823e-01  1.38463073e-01  1.39391783e-01  4.15408528e-01
 -2.36833813e-02 -1.45236900e-01 -3.39175533e-02 -1.70833024e+00
  1.38362405e+00  8.91049079e-01 -8.08974962e-02  6.87058115e-02
 -4.48806912e-01 -8.66663006e-02 -3.84102511e-01 -2.88558858e-02
 -3.80983773e-02  1.93883649e-02 -4.40846562e-01  1.56253354e-01
  8.25244024e-02  1.12472722e-01 -1.47811270e-01 -3.41628791e-01
 -1.08998973e-01 -4.72375678e-02  4.39088464e-01  1.76466709e-01
  1.4659802

#### VISUALIZATION OF GRAPH

In [324]:
# # Initialize the graph and add all nodes first
# G = nx.Graph()
# nodes = set()
# for (i, j) in Q.keys():
#     nodes.add(i)
#     nodes.add(j)
# G.add_nodes_from(nodes)

# # Now add edges and node attributes
# for (i, j), weight in Q.items():
#     if weight != 0 and i <= j:  # Avoid duplicates and zero-weights
#         if i == j:
#             G.nodes[i]['bias'] = weight  # Now safe because node exists
#         else:
#             G.add_edge(i, j, weight=weight)

# # Visualization
# plt.figure(figsize=(12, 10))
# pos = nx.circular_layout(G)

# # Draw nodes and edges
# nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=800)
# nx.draw_networkx_labels(G, pos, font_size=12)

# # Scale edge widths by absolute weight
# max_weight = max(abs(d['weight']) for _, _, d in G.edges(data=True))
# edge_widths = [0.1 + 2 * abs(G[u][v]['weight']) / max_weight for u, v in G.edges()]
# nx.draw_networkx_edges(G, pos, width=edge_widths, edge_color='gray')

# # Add edge labels (rounded)
# edge_labels = {(u, v): f"{d['weight']:.1f}" for u, v, d in G.edges(data=True)}
# nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

# # Add node biases
# for node, (x, y) in pos.items():
#     bias = G.nodes[node].get('bias', 0)
#     plt.text(x, y-0.1, f"bias={bias:.1f}", ha='center', fontsize=8, color='red')

# plt.title("QUBO Graph (Edge Width ∝ |Weight|)", fontsize=14)
# plt.axis('off')
# plt.show()

In [325]:
# pos = nx.spring_layout(G, k=0.5, weight='weight', iterations=100)
# nx.draw(G, pos, with_labels=True, 
#        node_color='lightblue', node_size=800,
#        width=[0.1 + 2*abs(G[u][v]['weight'])/max_weight for u,v in G.edges()],
#        edge_color='gray')

In [326]:
# from mpl_toolkits.mplot3d import Axes3D

# pos = nx.spring_layout(G, dim=3, weight='weight')
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# for u, v in G.edges():
#     ax.plot([pos[u][0], pos[v][0]], 
#             [pos[u][1], pos[v][1]], 
#             [pos[u][2], pos[v][2]], 
#             'gray')
# ax.scatter(*np.array(list(pos.values())).T, s=100, c='lightblue')
# plt.show()

#### FINAL EVALUATION

In [327]:
data = {
    "Dataset_Name": [dataset_name] * 6,
    "Number of Row": [df.shape[0]] * 6,  
    "Precision": [precision] * 6,
    "Degree": [degree] * 6,
    "Number_Of_Features": [d] * 6,
    "Model": ["Neal", "Neal", "Fixstar", "Fixstar", "Sklearn", "Sklearn"],
    "Dataset_split": ["Train", "Test", "Train", "Test", "Train", "Test"],
    "MSE": [
        train_mse_neal, test_mse_neal,
        train_mse_fixstar, test_mse_fixstar,
        train_mse_sklearn, test_mse_sklearn
    ],
    "MAE": [
        train_mae_neal, test_mae_neal,
        train_mae_fixstar, test_mae_fixstar,
        train_mae_sklearn, test_mae_sklearn
    ],
    "R2 Score": [
        train_r2_neal, test_r2_neal,
        train_r2_fixstar, test_r2_fixstar,
        train_r2_sklearn, test_r2_sklearn
    ],
    "preproccesing_time": [
        neal_preproccesing_time, neal_preproccesing_time,
        fixstar_preproccesing_time, fixstar_preproccesing_time,
        "-", "-" 
    ],
    "execution_time": [
        neal_execution_time, neal_execution_time,
        fixstar_execution_time, fixstar_execution_time,
        sklearn_execution_time, sklearn_execution_time
    ],
    "postproccesing_time": [
        neal_postproccesing_time, neal_postproccesing_time,
        fixstar_postproccesing_time, fixstar_postproccesing_time,
        "-", "-"
    ],
    "total_time": [
        neal_preproccesing_time + neal_execution_time + neal_postproccesing_time,neal_preproccesing_time + neal_execution_time + neal_postproccesing_time,
        fixstar_preproccesing_time + fixstar_execution_time + fixstar_postproccesing_time,fixstar_preproccesing_time + fixstar_execution_time + fixstar_postproccesing_time,
        sklearn_execution_time, sklearn_execution_time
    ]
}

df_eval = pd.DataFrame(data)
df_eval

,Dataset_Name,Number of Row,Precision,Degree,Number_Of_Features,Model,Dataset_split,MSE,MAE,R2 Score,preproccesing_time,execution_time,postproccesing_time,total_time
0,dataset_8.csv,2049280,3,4,210,Neal,Train,0.007323,0.052575,0.199627,7.876893,20.475179,0.064859,28.416931
1,dataset_8.csv,2049280,3,4,210,Neal,Test,0.007353,0.052661,0.201786,7.876893,20.475179,0.064859,28.416931
2,dataset_8.csv,2049280,3,4,210,Fixstar,Train,0.000105,0.007008,0.988521,10.232853,10.365872,0.002198,20.600924
3,dataset_8.csv,2049280,3,4,210,Fixstar,Test,0.000105,0.007013,0.988591,10.232853,10.365872,0.002198,20.600924
4,dataset_8.csv,2049280,3,4,210,Sklearn,Train,0.000010,0.001890,0.998960,-,29.798314,-,29.798314
5,dataset_8.csv,2049280,3,4,210,Sklearn,Test,0.000009,0.001887,0.998979,-,29.798314,-,29.798314


In [328]:
excel_filename = "evaluation_results2.xlsx"

# Add new data
append_to_excel_enhanced(
    excel_filename,
    df_eval,
    check_columns=['Dataset_Name', 'Precision', 'Degree']
)

Data successfully appended to Excel file.
